## ML News

- Orca LM: -> Not instruction - output pairs are important! -> Instruction -> Reasoning Chain/Explanation -> Output Pairs! = RULES OF LOGIC/SEMANTIC/REASONING
    - https://arxiv.org/pdf/2306.02707.pdf
    - huggingface is waiting ...: https://huggingface.co/papers/2306.02707
- LTM: 5 Milltion Tokens Context
    - JFI: Beginning 2022: 1024, Mid: 2023 -> December 2023: GPT-4: 8196 -> April: MPT: 65k -> Juni: LTM: 5M
    - https://magic.dev/blog/ltm-1
- Bard Logic Update: 
    - = REPL Tool integrated into Bard -> boost for logic tasks
    - https://blog.google/technology/ai/bard-improved-reasoning-google-sheets-export/
- SPQR Quantization: = almost LOSLESS 3-4 BIT Quantization is here!
    - https://arxiv.org/abs/2306.03078
    - possible next step: we have seen QLORA fine tuning -> can we do quantized training from scratch?


# Transformer Language Models

In [14]:
# PREPARE DATASET
## SHAKESPEARE DATSET

with open('./shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
## length of dataset
print("length of dataset in characters: ", len(text)) 
print('--------------')
 
## check first 500 characters
print(text[:500])
print('--------------')

## get all characters alphabetically sorted 
characters = sorted(list(set(text))) 
vocab_size = len(characters)
print(''.join(characters))
print(vocab_size)

# create a character to index mapping because it is easier to work with indices when using tensor matrices -> every character gets assigned an index
character_to_index_map = {character:index for index, character in enumerate(characters)}
print(character_to_index_map)
print('--------------')
# write characters into the cells to make it look more nicely
index_to_character_map = {index:character for character, index in character_to_index_map.items()}
print(index_to_character_map)

# TOKENIZER
## encoding/decoding functions to turn strings into lists of integers
encode = lambda s: [character_to_index_map[character] for character in s] # encoder takes a string, outputs a list of integers
decode = lambda l: ''.join([index_to_character_map[index] for index in l]) # decoder: take a list of integers, outputs a string     

print('--------------')

print(encode("hello world"))
print(decode(encode("hello world")))

length of dataset in characters:  1115393
--------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor
--------------

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 4

In [12]:
# DATA PREPROCESSING
import torch
 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500]) # the first 500 characters from the print above look to the model like this

# TRAIN/VAL SPLIT
training_percentage = 0.9
n = int(training_percentage * len(data))
train_data = data[:n]
val_data = data[n:]

# set context size
context_length = 8 

# build training examples for different context lengths starting from 1 to context_length
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1] # why t+1? -> slicing is exclusive in python
    target = y[t]
    print(f"when the input is {context} then we want to predict: {target}")
    
# idea here -> do next token prediction but for ALL context lengths up to max_context_length!
# why? -> because then we can start generation from single start token

# DATA SET COMPILATION
batch_size = 4
context_length = 8 # = maximum context length

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    indices = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in indices])
    y = torch.stack([data[i+1:i+context_length+1] for i in indices])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('--------------')
print('targets')
print(yb.shape)
print(yb)
print('--------------')

# test batch loading
for b in range(batch_size): # batch dimension 
    for t in range(context_length): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t] # why t here? -> because we shifted yb already 1 token! -> see above 
        print(f"when the input is {context.tolist()} then we want to predict: {target}")

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

## Transformer Architecture

---

![](https://heidloff.net/assets/img/2023/02/transformers.png)

---

![](https://upload.wikimedia.org/wikipedia/commons/9/91/Full_GPT_architecture.png)

---

#### The Illustrated Transformer
- https://jalammar.github.io/illustrated-transformer/

In [16]:
# TRANSFORMER NETWORK: https://arxiv.org/abs/1706.03762
from torch import nn
import torch
import torch.nn.functional as F

# Attention: attention is a communication mechanism where a number of nodes in a 
# connected graph. nodes aggregate information from all the nodes that point to it
# attention can be applied to any arbitrary connected graph
# attention has no notion of space -> thats why we need a positional encoding (if we want to have a notion of space)
# => this is different e.g. from convolution, where aggregation happens over space!

# optimizations for deep neural networks: residual connections/skip connections + layer normalization

class Transformer(nn.Module):
    def __init__(self, n_layers):
        super().__init__()
        self.embedding_layer = nn.Embedding(vocab_size, n_embed)
        self.position_embedding = nn.Embedding(context_length, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B, T) tensors of integers
        token_embeddings = self.embedding_layer(idx)
        pos_embeddings = self.position_embedding(torch.arange(T, device=device)) # (T, C)
        x = token_embeddings + pos_embeddings
        x = self.blocks(x) # apply blocks
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None: 
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last context_length tokens
            idx_cond = idx[:, -context_length:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # become (B, C) 
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
            

class AttentionHead(nn.Module):
    """Implements a single attention head""" 
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False) 
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B, T, C)
        q = self.query(x) # B, T, C)
        # compute attention scores
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T) -> C = embedding dimension -> divide by embedding dimension to keep the softmax stable, C = n_embed
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B, T, C) # v0*a0 + v1*a1 ....
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
    
class MultiHeadAttention(nn.Module):
    """implements multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, C)
        out = self.dropout(self.proj(out))
        return out
        
    
class FeedForward(nn.Module):
    """A sinple linear layer followed by a non-linearity"""
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4* n_embed), # ffwd network
            nn.ReLU(),
            nn.Linear(4* n_embed, n_embed), # projection network
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """Transformer block. First multi head attention for communication then feed forward network for computation"""
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.self_attention = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed) # = per token normalization -> makes the inputs be normal distributed -> stabilizes training
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self, x):
        x = x + self.self_attention(self.ln1(x)) # residual connection -> this deviates to original -> we apply layernorm before self attention and before ffwd -> in paper vaswani does it afterwards
        x = x + self.ffwd(self.ln2(x)) # residual connection
        # layer norm -> normalizes activations to have zero mean and std 1 over all inputs
        return x 
    
class LayerNorm():
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True) # input mean
        xvar = x.var(1, keepdim=True) # input variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma + xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
def estimate_losses(model):
    # estimate train loss
    xb, yb = get_batch('train')
    _, train_loss = model(xb, yb)
    # estimate val loss
    xb, yb = get_batch('train')
    _, val_loss = model(xb, yb)
    losses = {'train': train_loss, 'val': val_loss}
    return losses

In [17]:
# TRAINING
from torch import nn
import torch

batch_size = 4
context_length = 64
max_iters = 2000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384 # -> 384//6 = 64 -> every attention head is 64 dimensional
n_heads = 6 # number of heads in multi head attention
n_layers = 2
dropout = 0.2 # 20 percent of connections are masked out randomly

model = Transformer(n_layers=n_layers)
# put model on GPU, if you have one ;)
#model = model.to(device)

# create optimizer  
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# TRAINING LOOP
for iteration in range(max_iters):
    # evaluation
    if (iteration % eval_interval == 0 or iteration == max_iters-1):
        losses = estimate_losses(model)
        print(f"step {iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
    # sample a batch of data
    xb, yb = get_batch('train')
     
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3344, val loss 4.4027
step 100: train loss 2.8727, val loss 2.6931
step 200: train loss 2.5597, val loss 2.4263
step 300: train loss 2.5942, val loss 2.5781
step 400: train loss 2.3385, val loss 2.5185
step 500: train loss 2.3910, val loss 2.3671
step 600: train loss 2.3267, val loss 2.2162
step 700: train loss 2.3075, val loss 2.4056
step 800: train loss 2.1323, val loss 2.3248
step 900: train loss 2.2218, val loss 2.1749
step 1000: train loss 2.1252, val loss 2.2314
step 1100: train loss 2.2216, val loss 2.0954
step 1200: train loss 2.0816, val loss 2.1144
step 1300: train loss 2.1495, val loss 2.1939
step 1400: train loss 2.1802, val loss 2.1289
step 1500: train loss 2.0668, val loss 2.0967
step 1600: train loss 2.1368, val loss 2.0730
step 1700: train loss 2.1487, val loss 2.0349
step 1800: train loss 2.0953, val loss 2.2398
step 1900: train loss 1.9716, val loss 2.0479
step 1999: train loss 1.9992, val loss 2.0144


In [18]:
# SAMPLING

context = torch.zeros((1,1), dtype=torch.long, device=device) # this is the new line character \n -> index 0, batch size 1 -> start generating from the newline character
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


GLA:
Yold he woaker me ne Wis hirs you,
Why that sword he rusd thom bowath' hat'd wicl
I Venevry:
Frster, are it sas by brain. O!

Thy, OF Yhust.

KING RIING RIS GICHARGELIET:
Havebest bill wam I ging atheige
That phe cane theimcy,
Son's tir-for them Offech and mainks, ay.

Sade snould gragt RIOH:
Agve with rence son;
Lath be kin, you ispeclor wifl crvest; no pring?

Muchtme in therable's not hid cur:
Hat If hal be tostle maser.

HUMEThe:
Comenst shond deexir mand my plagained gon
But ondremin t
